In [2]:
import numpy as np
import pandas as pd
import recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
import geo_utils
from constants import geocat_constants
from constants import experiment_constants
from constants import usg_constants
from collections import defaultdict
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
%matplotlib inline

CITY=experiment_constants.get_city()

In [2]:
df_city_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})
users_id=df_city_checkin['user_id'].drop_duplicates().reset_index(drop=True)

# USG uses unique checkins cij = 1 or cij = 0
df_city_checkin=df_city_checkin.drop_duplicates(subset=['user_id','business_id']).reset_index(drop=True)

df_city_checkin=df_city_checkin.set_index('user_id')
df_city_checkin.head()

# U: preference

$CosineSimilarity = cos(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}$

In [4]:
def user_similarity(user_checkin_1,user_checkin_2):
    common_checkin=pd.merge(user_checkin_1,user_checkin_2,on='business_id')
    return common_checkin.date_x.count()/(math.sqrt(user_checkin_1.date.count())
                                   *math.sqrt(user_checkin_2.date.count()))

# user_similarity(df_city_checkin.loc['46TNf-5T5EcNFRJPYv_nrw'],
#                 df_city_checkin.loc['56iEnLi8jR--2ranjPSQ4w'])

$\#df\_user\_similarity=\frac{\#usuarios\cdot (\#usuarios-1)}{2}=\sum_{i=1}^{\#usuarios}{i}$

In [5]:
print(users_id.count(),users_id.count()*(users_id.count()-1)/2)

1022 521731.0


In [6]:
df_user_similarity=pd.DataFrame(columns=['user_id_1','user_id_2','similarity'])
i=0
start = timeit.default_timer()
users_id_removable=users_id.copy()
# loop sobre os ids dos usuarios
for index_1,user_id_1 in users_id.iteritems():
    # lista de checkins do usuario
    df_user_checkin_1=df_city_checkin.loc[[user_id_1]]
    # Exclui o proprio usuario
    users_id_removable.drop(index_1,inplace=True)
    for index_2,user_id_2 in users_id_removable.iteritems():
#         if len(df_user_similarity[((df_user_similarity.user_id_1==user_id_1) | (df_user_similarity.user_id_1==user_id_2)) &
#                         ((df_user_similarity.user_id_2==user_id_1) | (df_user_similarity.user_id_2==user_id_2))])!=0:
#             continue
            
        df_user_checkin_2=df_city_checkin.loc[[user_id_2]]
        #print(df_user_checkin_1)
        sim=user_similarity(df_user_checkin_1,df_user_checkin_2)
        df_user_similarity=df_user_similarity.append({'user_id_1':user_id_1,'user_id_2':user_id_2,'similarity':sim},ignore_index=True)
    
    df_city_checkin=df_city_checkin.loc[users_id_removable]
    
    if i % 10 == 0:
        print(i/users_id.count())
    i=i+1

stop = timeit.default_timer()
print('Time:', stop - start)
df_user_similarity

0.0
0.009784735812133072
0.019569471624266144
0.029354207436399216
0.03913894324853229
0.04892367906066536
0.05870841487279843
0.0684931506849315
0.07827788649706457
0.08806262230919765
0.09784735812133072
0.10763209393346379
0.11741682974559686
0.12720156555772993
0.136986301369863
0.14677103718199608
0.15655577299412915
0.16634050880626222
0.1761252446183953
0.18590998043052837
0.19569471624266144
0.2054794520547945
0.21526418786692758
0.22504892367906065
0.23483365949119372
0.2446183953033268
0.25440313111545987
0.26418786692759294
0.273972602739726
0.2837573385518591
0.29354207436399216
0.30332681017612523
0.3131115459882583
0.32289628180039137
0.33268101761252444
0.3424657534246575
0.3522504892367906
0.36203522504892366
0.37181996086105673
0.3816046966731898
0.3913894324853229
0.40117416829745595
0.410958904109589
0.4207436399217221
0.43052837573385516
0.44031311154598823
0.4500978473581213
0.4598825831702544
0.46966731898238745
0.4794520547945205
0.4892367906066536
0.499021526418

,user_id_1,user_id_2,similarity
0,MB7Z80PJxNyQM7keDpR2XQ,4HwS2oCVQ1-f_kzHCv5X3A,0.093891
1,MB7Z80PJxNyQM7keDpR2XQ,OLBH2Ds1wllrzpHlyojzJQ,0.234509
2,MB7Z80PJxNyQM7keDpR2XQ,wd3xoNaDLib8dhQ7BxUl6g,0.108556
3,MB7Z80PJxNyQM7keDpR2XQ,qp-z1copnYQq7NPubYlNLw,0.018452
4,MB7Z80PJxNyQM7keDpR2XQ,e9bVxMuG-1NIJWqT_pDEmQ,0.000000
5,MB7Z80PJxNyQM7keDpR2XQ,SDsVaBzPJr8fdY4-m5bjdw,0.000000
6,MB7Z80PJxNyQM7keDpR2XQ,tBZ8UAQFooesK_kst774ug,0.094407
7,MB7Z80PJxNyQM7keDpR2XQ,tbycp0aRhRsiAVYUAVR_uQ,0.071679
8,MB7Z80PJxNyQM7keDpR2XQ,bf4JWlkcOf_UTwhAumrIJg,0.000000
9,MB7Z80PJxNyQM7keDpR2XQ,w7FqbR6nPEpyeqy-mYhfcQ,0.047378


In [7]:
df_user_similarity.to_csv("../data/usg/user_similarity/"+CITY+".csv",index=False)

In [ ]:
def user_cf(user_id,user_checkin,business_id,df_checkin):
    df_business_checkin=df_checkin[df_checkin['business_id']==business_id]
    df_checkin=df_checkin[df_city_checkin['user_id']!=user_id]
    return df_checkin.apply(lambda checkin: user_similarity(user_checkin,df_checkin[df_checkin['user_id']==checkin['user_id']]),axis=1)

user_cf('46TNf-5T5EcNFRJPYv_nrw',df_city_checkin[df_city_checkin.user_id=='46TNf-5T5EcNFRJPYv_nrw'],'ujmEBvifdJM6h6RLv4wQIg',df_city_checkin)

In [ ]:
for index,user_id in users_id.iteritems():
    print(user_id)
    df_user_checkin=df_city_checkin[df_city_checkin[.index]==user_id]
    print(df_user_checkin)
    break

# S: social

In [7]:
df_city_user=pd.read_csv("../data/user/"+CITY+".csv",converters={'friends':string_to_array})

In [8]:
df_city_user.set_index('user_id',inplace=True)

df_city_user.head()

,friends
user_id,
scaxsfOynKUZEijn_UCTkg,"[XmUoErm8KkfP_ti71nXgAg, JOd9BEaOuPRGEc5LOFB6M..."
CY9iLsE2z_yLhLqJdD1WGw,"[5oM7gXmeX7s6T1_KlL30yQ, wm61ALFiZ-7BCbX57evHP..."
HNycc5PctOiR2PXb-TFCIQ,"[btwW4jGa1zFCBu5q5cCnEg, BVHZ6w_YgA1lt8Rnq4NSC..."
claV_wXYEgojvBrRodrSGQ,"[b-0Kk5zzCdJuU30xAVzZdQ, wd3xoNaDLib8dhQ7BxUl6..."
XK1Qd9M8XjgoqT1FLfW6bQ,"[d5WLqmTMvmL7-RmUDVKqqQ, XmUoErm8KkfP_ti71nXgA..."


$SI_{k,i}=\eta\cdot\frac{|F_k\cap F_i|}{|F_k\cup F_i|} + (1-\eta)\cdot\frac{|L_k\cap L_i|}{|L_k\cup L_i|}$

$F_k:$ is the friend set of user k

$L_k:$ denote the POI set of user k

$\eta:$ tuning parameter, smaller than 0.05 gives optimal result.(optimal setting)

In [9]:
def social_influence(df_friends_1,df_friends_2,df_user_poi_1,df_user_poi_2,tuning_parameter):
    common_friends=len(pd.merge(df_friends_1,df_friends_2))
    union_friends=len(pd.concat([df_friends_1,df_friends_2]).drop_duplicates())
    common_pois=len(pd.merge(df_user_poi_1,df_user_poi_2))
    union_pois=len(pd.concat([df_user_poi_1,df_user_poi_2]).drop_duplicates())
    return tuning_parameter*common_friends/union_friends+(1-tuning_parameter)*common_pois/union_pois
# df_friends_1=df_city_user.loc['56iEnLi8jR--2ranjPSQ4w']
# df_friends_2=df_city_user.loc['Nw6Jza_cb0aqd4PT769Vzw']

# df_friends_1=pd.DataFrame(df_friends_1['friends'],columns=['user_id'])
# df_friends_2=pd.DataFrame(df_friends_2['friends'],columns=['user_id'])
# df_user_poi_1=df_city_checkin.loc['56iEnLi8jR--2ranjPSQ4w']['business_id'].drop_duplicates()
# df_user_poi_2=df_city_checkin.loc['Nw6Jza_cb0aqd4PT769Vzw']['business_id'].drop_duplicates()


# social_influence(df_friends_1,df_friends_2,df_user_poi_1,df_user_poi_2,usg_constants.get_eta())

In [10]:
df_city_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})
users_id=df_city_checkin['user_id'].drop_duplicates().reset_index(drop=True)
# USG uses unique checkins cij = 1 or cij = 0
df_city_checkin=df_city_checkin.drop_duplicates(subset=['user_id','business_id']).reset_index(drop=True)
df_city_checkin=df_city_checkin.set_index('user_id')
df_city_checkin.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
MB7Z80PJxNyQM7keDpR2XQ,br1uQbc3YU5jCpYi46nIgQ,2018-04-04 00:09:03,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0
4HwS2oCVQ1-f_kzHCv5X3A,br1uQbc3YU5jCpYi46nIgQ,2018-02-16 20:40:41,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0
OLBH2Ds1wllrzpHlyojzJQ,br1uQbc3YU5jCpYi46nIgQ,2018-04-12 11:31:03,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0
wd3xoNaDLib8dhQ7BxUl6g,br1uQbc3YU5jCpYi46nIgQ,2018-03-07 14:08:41,43.069425,-89.389567,"[Coffee & Tea, Cafes]",2064.0
qp-z1copnYQq7NPubYlNLw,uLnPW9K1yhNpgzvodCumbQ,2014-10-27 04:19:36,42.979252,-89.516184,"[Breweries, Beer Bar, Bars, Burgers, American ...",684.0


In [12]:
df_user_social_influence=pd.DataFrame(columns=['user_id_1','user_id_2','social_influence'])
num_users=len(users_id)


start = timeit.default_timer()
for index_1,user_id_1 in users_id.iteritems():
    # lista de checkins do usuario
    #df_user_checkin_1=df_city_checkin[df_city_checkin.user_id==user_id_1]
    #df_city_user
    friends_list=df_city_user.loc[user_id_1]['friends']
    df_friends_1=pd.DataFrame(friends_list,columns=['user_id'])
    #print(df_city_checkin)
    df_user_poi_1=df_city_checkin.loc[user_id_1]['business_id'].drop_duplicates()
    for user_id_2 in friends_list:
        df_friends_2=pd.DataFrame(df_city_user.loc[user_id_2]['friends'],columns=['user_id'])
        df_user_poi_2=df_city_checkin.loc[user_id_2]['business_id'].drop_duplicates()
        si=social_influence(df_friends_1,df_friends_2,df_user_poi_1,df_user_poi_2,usg_constants.get_eta())
        df_user_social_influence=df_user_social_influence.append({'user_id_1':user_id_1,'user_id_2':user_id_2,'social_influence':si},ignore_index=True)
    if index_1%125==0:
        print(str(100*index_1/num_users)+"%")

stop = timeit.default_timer()
print('Time:', (stop - start)/60,'minutes')

0.0%
12.230919765166341%
24.461839530332682%
36.69275929549902%
48.923679060665364%
61.154598825831705%
73.38551859099805%
85.61643835616438%
97.84735812133073%
Time: 1.746310833733393 minutes


In [15]:
df_user_social_influence.to_csv("../data/usg/social_influence/"+CITY+".csv",index=False)
df_user_social_influence.head(1)

,user_id_1,user_id_2,social_influence
0,MB7Z80PJxNyQM7keDpR2XQ,Ff3tzelMfkB033i8s3yGvg,0.016323


In [ ]:
df_city_user.head()

In [ ]:
i=0
df_city_checkin[df_city_checkin.index=='46TNf-5T5EcNFRJPYv_nrw']
for index,checkin in df_city_checkin.iteritems():
    print(index)
    print(checkin)
    if i%22==0:
        print(i)
        break
    i=i+1

# G: geographical



$y'(x',w)=w_0+w_1\cdot x'$

In [13]:
def checkin_probability(a,b,dist_km):
    get_dist_new_unit(dist_km) # 100m unit
    probability=a*(dist_km**b)
    return min(probability,1.0)
    
def get_dist_new_unit(dist_km):
    dist_new_unit=dist_km*10
    dist_new_unit=max(dist_new_unit,1.0)
    dist_new_unit=dist_new_unit+1
    return dist_new_unit


In [ ]:

def compute_distance_distribution(df_city_checkin):
    ic=0
    distribution = defaultdict(int)
    for user_id in users_id:
        if ic % 100 == 0:
            print(i)
        ic=ic+1
        #print(user_id)
        df_user_checkins=df_city_checkin.loc[user_id]

        #print("quantidade de checkins do usuario:",len(df_user_checkins),", combinações:",len(df_user_checkins)*(len(df_user_checkins)-1)/2)
    #    print(df_user_checkins)
        for i in range(len(df_user_checkins)):
            for j in range(i+1,len(df_user_checkins)):
                lat1,lon1=df_user_checkins.iloc[i].latitude,df_user_checkins.iloc[i].longitude
                lat2,lon2=df_user_checkins.iloc[j].latitude,df_user_checkins.iloc[j].longitude
                distance=int(geo_utils.mercator(lat1,lon1,lat2,lon2))
                distribution[distance] += 1
    total=sum(distribution.values())
    for distance in distribution:
        distribution[distance] /= total
    distribution=sorted(distribution.items(),key=lambda k: k[0])
    return zip(*distribution[1:]) # x and ground truth



start = timeit.default_timer()
x,t=compute_distance_distribution(df_city_checkin[['business_id','longitude','latitude']])
stop = timeit.default_timer()
print('Time:', stop - start)

0
100
100


In [ ]:
x=np.log10(x)
t=np.log10(t)

w0, w1 = np.random.random(), np.random.random()
max_iterations = 2000
lambda_w = 0.1
alpha = 1e-5 # learning rate
for iteration in range(max_iterations):
    Ew = 0.0
    d_w0, d_w1 = 0.0, 0.0
    for n in range(len(x)):
        d_w0 += (w0 + w1 * x[n] - t[n])
        d_w1 += (w0 + w1 * x[n] - t[n]) * x[n]
    w0 -= alpha * (d_w0 + lambda_w * w0)
    w1 -= alpha * (d_w1 + lambda_w * w1)
    for n in range(len(x)):
        Ew += 0.5 * (w0 + w1 * x[n] - t[n])**2
    Ew += 0.5 * lambda_w * (w0**2 + w1**2)

a,b=10**w0,w1



